In [2]:
import numpy as np
from lab3_tools import *
import sys
sys.path.insert(0, '../lab1/amrita')
sys.path.insert(1, '../lab2/')
from lab1_proto import mfcc,mspec


ModuleNotFoundError: No module named 'lab3_tools'

load the emitting states in the phoneHMMs models from Lab 2 as target
classes for the deep neural networks.

In [5]:
phoneHMMs = np.load('../lab2/lab2_models_all.npz',allow_pickle=True)['phoneHMMs'].item()
phones = sorted(phoneHMMs.keys())
nstates = {phone: phoneHMMs[phone]['means'].shape[0] for phone in phones}
stateList = [ph + '_' + str(id) for ph in phones for id in range(nstates[ph])]
# np.save('StateList',stateList)

the numerical index of a particular state can be obtained by

In [6]:
example = np.load('data/lab3_example.npz',allow_pickle=True)['example'].item()

In [7]:
example.keys()

dict_keys(['filename', 'samples', 'gender', 'speaker', 'digits', 'repetition', 'lmfcc', 'wordTrans', 'phoneTrans', 'utteranceHMM', 'stateTrans', 'obsloglik', 'viterbiLoglik', 'viterbiPath', 'viterbiStateTrans'])

In [8]:
stateList=list(np.load('data/StateList.npy'))
stateList.index('ay_2')

8

In [9]:
filename = 'data/tidigits/disc_4.1.1/tidigits/train/man/nw/z43a.wav'
samples, samplingrate = loadAudio(filename)
print('Samples verified:',np.array_equal(example['samples'],samples))
lmfcc = mfcc(samples,)
print('MFCC verified:',np.allclose(example['lmfcc'],lmfcc))

Samples verified: True
MFCC verified: True


In [10]:
wordTrans = list(path2info(filename)[2])

print('wordTrans verified:',np.array_equal(example['wordTrans'],wordTrans))


wordTrans verified: True


In [11]:
def words2phones(wordList, pronDict, addSilence=True, addShortPause=True):
    """ word2phones: converts word level to phone level transcription adding silence

    Args:
       wordList: list of word symbols
       pronDict: pronunciation dictionary. The keys correspond to words in wordList
       addSilence: if True, add initial and final silence
       addShortPause: if True, add short pause model "sp" at end of each word
    Output:
       list of phone symbols
    """
    phoneSymbols=['sil']
    for word in wordList:
        phoneSymbols+=pronDict[word]
        phoneSymbols.append('sp')
    phoneSymbols.append('sil')
    return phoneSymbols


In [12]:
from prondict import prondict
phoneTrans = words2phones(wordTrans, prondict)
print('phoneTrans verified:',np.array_equal(example['phoneTrans'],phoneTrans))

phoneTrans verified: True


In [13]:
from lab2_proto import concatHMMs
utteranceHMM = concatHMMs(phoneHMMs, phoneTrans)
print('utteranceHMM verified :')
print('startprob verified :',np.array_equal(utteranceHMM['startprob'],example['utteranceHMM']['startprob']))
print('transmat verified :',np.array_equal(utteranceHMM['transmat'],example['utteranceHMM']['transmat']))
print('means verified :',np.array_equal(utteranceHMM['means'],example['utteranceHMM']['means']))
print('covars verified :',np.array_equal(utteranceHMM['covars'],example['utteranceHMM']['covars']))

utteranceHMM verified :
startprob verified : True
transmat verified : True
means verified : True
covars verified : True


In [14]:
phoneHMMs['sp']

{'name': 'sp',
 'startprob': array([0.1216644, 0.8783357]),
 'transmat': array([[0.8656793, 0.1343206],
        [0.       , 1.       ]]),
 'means': array([[  38.44177  , -129.2183   ,   54.57983  ,   15.29446  ,
           35.78878  ,   -6.647676 ,  -11.7913   ,  -25.44149  ,
          -20.31962  ,  -27.77863  ,   -5.970622 ,    0.9313969,
           22.0168   ]]),
 'covars': array([[ 1672.14 ,  2563.211,  2475.374,  3410.646,  4425.932,  5530.672,
          7249.499,  9209.617, 10143.23 ,  9683.029,  8902.944,  8219.629,
          7589.251]])}

In [15]:
stateTrans = [phone + '_' + str(stateid) for phone in phoneTrans for stateid in range(nstates[phone])]
print('stateTrans verified :',np.array_equal(stateTrans,example['stateTrans']))

stateTrans verified : True


In [16]:
from lab2_tools import log_multivariate_normal_density_diag


In [17]:
obsloglik=log_multivariate_normal_density_diag(lmfcc,utteranceHMM['means'],utteranceHMM['covars'])
print('obsloglik verified :',np.allclose(obsloglik,example['obsloglik']))

obsloglik verified : True


In [18]:
def viterbi(log_emlik, log_startprob, log_transmat, forceFinalState=True):
    """Viterbi path.
    Args:
        log_emlik: NxM array of emission log likelihoods, N frames, M states
        log_startprob: log probability to start in state i
        log_transmat: transition log probability from state i to j
        forceFinalState: if True, start backtracking from the final state in
                  the model, instead of the best state at the last time step
    Output:
        viterbi_loglik: log likelihood of the best path
        viterbi_path: best path
    """
    N, M = log_emlik.shape
    B = np.zeros((N, M))
    V = np.zeros((N, M))
    V[0] = np.log(log_startprob) + log_emlik[0]

    for t in range(1, N):
        for j in range(M):
            V[t][j] = np.max(V[t-1] + np.log(log_transmat[:,j])) + log_emlik[t][j]
            B[t][j] = np.argmax(V[t-1] + np.log(log_transmat[:,j]))
    
    best_score = np.max(V[t])
    
    path = np.zeros(N)
    path[t] = np.argmax(B[t, :])

    for t in range(N-2, 0, -1):
        path[t] = B[t+1,int(path[t+1])]
    
    return best_score,path

In [19]:
_,viterbiPath=viterbi(obsloglik,utteranceHMM['startprob'][:-1],utteranceHMM['transmat'][:-1,:-1])
viterbiPath = list(map(int, viterbiPath))
print('viterbiPath verified :',np.array_equal(viterbiPath,example['viterbiPath']))

viterbiPath verified : True


In [20]:
stateTrans=np.asarray(stateTrans)
viterbiStateTrans=stateTrans[viterbiPath]
print('viterbiStateTrans verified :',np.array_equal(viterbiStateTrans,np.asarray(example['viterbiStateTrans'])))

viterbiStateTrans verified : True


In [21]:
frames2trans(stateList, outfilename='z43a.lab')

'0 0.01 ah_0\n0.01 0.02 ah_1\n0.02 0.03 ah_2\n0.03 0.04 ao_0\n0.04 0.05 ao_1\n0.05 0.060000000000000005 ao_2\n0.060000000000000005 0.07 ay_0\n0.07 0.08 ay_1\n0.08 0.09 ay_2\n0.09 0.09999999999999999 eh_0\n0.09999999999999999 0.10999999999999999 eh_1\n0.10999999999999999 0.11999999999999998 eh_2\n0.11999999999999998 0.12999999999999998 ey_0\n0.12999999999999998 0.13999999999999999 ey_1\n0.13999999999999999 0.15 ey_2\n0.15 0.16 f_0\n0.16 0.17 f_1\n0.17 0.18000000000000002 f_2\n0.18000000000000002 0.19000000000000003 ih_0\n0.19000000000000003 0.20000000000000004 ih_1\n0.20000000000000004 0.21000000000000005 ih_2\n0.21000000000000005 0.22000000000000006 iy_0\n0.22000000000000006 0.23000000000000007 iy_1\n0.23000000000000007 0.24000000000000007 iy_2\n0.24000000000000007 0.25000000000000006 k_0\n0.25000000000000006 0.26000000000000006 k_1\n0.26000000000000006 0.2700000000000001 k_2\n0.2700000000000001 0.2800000000000001 n_0\n0.2800000000000001 0.2900000000000001 n_1\n0.2900000000000001 0.300

In [22]:
from lab1_proto import mspec
from tqdm import tqdm

In [23]:
stateList=list(np.load('data/StateList.npy'))
targets=[]
for i in viterbiStateTrans:
    # print(i)
    targets.append(stateList.index(i))

In [197]:
traindata = []
for root, dirs, files in os.walk('data/tidigits/disc_4.1.1/tidigits/train'):
    for file in tqdm(files):
        if file.endswith('.wav'):
            filename = os.path.join(root, file)
            samples, samplingrate = loadAudio(filename)
            lmfcc = mfcc(samples)
            wordTrans = list(path2info(filename)[2])
            phoneTrans = words2phones(wordTrans, prondict)
            utteranceHMM = concatHMMs(phoneHMMs, phoneTrans)
            obsloglik=log_multivariate_normal_density_diag(lmfcc,utteranceHMM['means'],utteranceHMM['covars'])
            _,viterbiPath=viterbi(obsloglik,utteranceHMM['startprob'][:-1],utteranceHMM['transmat'][:-1,:-1])
            viterbiPath = list(map(int, viterbiPath))
            stateTrans = [phone + '_' + str(stateid) for phone in phoneTrans for stateid in range(nstates[phone])]
            stateTrans=np.asarray(stateTrans)
            viterbiStateTrans=stateTrans[viterbiPath]
            targets=[]
            for i in viterbiStateTrans:
                targets.append(stateList.index(i))
            mspec_=mspec(samples)
            # stateList=stateTrans[viterbiStateTrans]
            traindata.append({'filename': filename, 'lmfcc': lmfcc,'mspec': mspec_, 'targets': targets})

0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 77/77 [00:25<00:00,  3.05it/s]
0it [00:00, ?it/s]
100%|██████████| 77/77 [00:26<00:00,  2.89it/s]


In [199]:
np.savez('data/train_data.npz', traindata)

In [203]:
testdata = []
for root, dirs, files in os.walk('data/tidigits/disc_4.2.1/tidigits/test'):
    for file in tqdm(files):
        if file.endswith('.wav'):
            filename = os.path.join(root, file)
            samples, samplingrate = loadAudio(filename)
            lmfcc = mfcc(samples)
            wordTrans = list(path2info(filename)[2])
            phoneTrans = words2phones(wordTrans, prondict)
            utteranceHMM = concatHMMs(phoneHMMs, phoneTrans)
            obsloglik=log_multivariate_normal_density_diag(lmfcc,utteranceHMM['means'],utteranceHMM['covars'])
            _,viterbiPath=viterbi(obsloglik,utteranceHMM['startprob'][:-1],utteranceHMM['transmat'][:-1,:-1])
            viterbiPath = list(map(int, viterbiPath))
            stateTrans = [phone + '_' + str(stateid) for phone in phoneTrans for stateid in range(nstates[phone])]
            stateTrans=np.asarray(stateTrans)
            viterbiStateTrans=stateTrans[viterbiPath]
            targets=[]
            for i in viterbiStateTrans:
                targets.append(stateList.index(i))
            mspec_=mspec(samples)
            # stateList=stateTrans[viterbiStateTrans]
            testdata.append({'filename': filename, 'lmfcc': lmfcc,'mspec': mspec_, 'targets': targets})

0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 77/77 [00:55<00:00,  1.39it/s]
0it [00:00, ?it/s]
100%|██████████| 77/77 [00:21<00:00,  3.58it/s]


In [204]:
np.savez('data/testdata.npz', testdata)

In [ ]:
np.savez('data/ters')

In [1]:
train_val=np.load('data/traindata.npz',allow_pickle=True)['arr_0']
test_data=np.load('data/testdata.npz',allow_pickle=True)['arr_0']

NameError: name 'np' is not defined

In [3]:
train_val.shape

(8623,)

In [4]:
validation = []
train = []
for x in train_val:
    if x['filename'].endswith('b.wav') and 'woman' in x['filename']:
        validation.append(x)
    elif x['filename'].endswith('b.wav') and 'man' in x['filename'] :
        validation.append(x)
    else:
        train.append(x)

In [5]:
print('Train size:',len(train))
print('Validation size:',len(validation))

Train size: 7373
Validation size: 1250


### 4.4 Split data into train and val set, stratify by gender & 4.5 Acoustic Context

In [3]:
import numpy as np
import pandas as pd

In [30]:
train_val=np.load('data/traindata.npz',allow_pickle=True)['arr_0']

In [31]:
train_val[0].keys()

dict_keys(['filename', 'lmfcc', 'mspec', 'targets'])

In [32]:
# 4.5 for each utterance and time step, stack 7 MFCC/filterbank features symmetrically distributed around current timestep
n = 14
for i in range(n):
    print(i)
    if i < 3:
        indices = [j for j in range(3-i,0,-1)]
        indices += [y for y in range(7-len(indices))]
    elif i > n-4:
        indices = [j for j in range(i-3, n)]
        indices += [y for y in range(n-2,n-2-(7-len(indices)),-1)]
    else:
        indices = [x for x in range(i-3,i+4)]
    print(indices)


0
[3, 2, 1, 0, 1, 2, 3]
1
[2, 1, 0, 1, 2, 3, 4]
2
[1, 0, 1, 2, 3, 4, 5]
3
[0, 1, 2, 3, 4, 5, 6]
4
[1, 2, 3, 4, 5, 6, 7]
5
[2, 3, 4, 5, 6, 7, 8]
6
[3, 4, 5, 6, 7, 8, 9]
7
[4, 5, 6, 7, 8, 9, 10]
8
[5, 6, 7, 8, 9, 10, 11]
9
[6, 7, 8, 9, 10, 11, 12]
10
[7, 8, 9, 10, 11, 12, 13]
11
[8, 9, 10, 11, 12, 13, 12]
12
[9, 10, 11, 12, 13, 12, 11]
13
[10, 11, 12, 13, 12, 11, 10]


In [6]:
def get_window_indices(i, n):
    if i < 3:
        indices = [j for j in range(3-i,0,-1)]
        indices += [y for y in range(7-len(indices))]
    elif i > n-4:
        indices = [j for j in range(i-3, n)]
        indices += [y for y in range(n-2,n-2-(7-len(indices)),-1)]
    else:
        indices = [x for x in range(i-3,i+4)]
    assert len(indices) == 7
    return indices


def process_data(data, window_features=False):
    ''' takes in data loaded from npz file and returns dataframe of features and utteance info '''
    filenames = []
    lmfccs = []
    mspecs = []
    targets = []
    speakers = []
#     utterances = [] # e.g. 'a'
#     repetitions = [] # e.g. a, b,... the version of the utterance spoken by the speaker
    genders = []

    ''' loop over points in train_val set and store data in arrays above'''
    for x in data:
        filename = x['filename']

        # get subject and gender
        f = filename.split('/')[-1]
        _, gender, speaker, utterance = f.split('\\')

        genders.append(gender)
        speakers.append(speaker)
        filenames.append(filename)
#         print(x['lmfcc'].shape)
        lmfccs.append(x['lmfcc'])
        mspecs.append(x['mspec'])
        targets.append(x['targets'])
    # 4.5 
    print('Computing dynamic features')
    n = len(lmfccs)
    lmfcc_windowed = []
    mspec_windowed = []
    
    # dynamic features for lmfccs
    for i in range(n):
        lmfcc_len = len(lmfccs[i])
        l_utterance = []
        for j in range(lmfcc_len):
            indices = get_window_indices(j,lmfcc_len) 
            l_frame = [lmfccs[i][x] for x in indices]
            # flatten
            l_frame = np.array([y for x in l_frame for y in x])
            l_utterance.append(l_frame) 
        # append to full feature set
        lmfcc_windowed.append(l_utterance)
    print(lmfcc_windowed[0][0].shape)
    #  dynamic features for mspec    
    for i in range(n):
        mspec_len = len(mspecs[i])
        m_utterance = []
        for j in range(mspec_len):
            indices = get_window_indices(j, mspec_len) 
            m_frame = np.array([mspecs[i][x] for x in indices])
            m_frame = np.array([y for x in m_frame for y in x])
            m_utterance.append(m_frame)
        # append to full feature set
        mspec_windowed.append(m_utterance)


    df = pd.DataFrame({'filename': filenames, 'speaker': speakers, 'gender': genders, \
                   'lmfcc_dynamic': lmfcc_windowed, 'mspec_dynamic': mspec_windowed, \
                           'lmfcc': lmfccs, 'mspec': mspecs, 'target': targets})
    
    
    return df

In [34]:
df_train = process_data(train_val)

Computing dynamic features
(91,)


In [35]:
df_train['lmfcc_dynamic'][1][0].shape

(91,)

In [36]:
df_train['lmfcc'][1][0].shape

(13,)

In [37]:
df_train['mspec_dynamic'][1][0].shape

(280,)

In [38]:
df_train['mspec'][1][0].shape

(40,)

In [39]:
df_train.shape

(8623, 8)

In [13]:
df_test.shape

NameError: name 'df_test' is not defined

Split df_train into train and val set, keeping speakers in different sets and stratifying by gender

In [40]:
np.unique(df_train['speaker'], return_counts=True)

(array(['ac', 'ae', 'ag', 'ai', 'aj', 'al', 'an', 'aw', 'bd', 'bh', 'bi',
        'br', 'ca', 'cb', 'cf', 'cg', 'cl', 'cm', 'cr', 'dc', 'dg', 'dl',
        'dn', 'ea', 'ec', 'ee', 'eg', 'eh', 'ei', 'ek', 'el', 'es', 'fc',
        'fd', 'ff', 'fi', 'fj', 'fk', 'fl', 'gg', 'gj', 'gr', 'gt', 'ha',
        'hg', 'hh', 'hl', 'hn', 'hp', 'hs', 'ie', 'if', 'ig', 'ih', 'il',
        'in', 'it', 'jc', 'ji', 'jj', 'jn', 'jp', 'jr', 'jt', 'kc', 'kd',
        'kf', 'kh', 'kk', 'kn', 'kp', 'kr', 'kt', 'la', 'ld', 'lh', 'ls',
        'lt', 'mb', 'mh', 'mk', 'mm', 'mp', 'mr', 'ms', 'mw', 'nc', 'ne',
        'nf', 'ng', 'nh', 'ni', 'nr', 'nw', 'pb', 'pd', 'pe', 'pi', 'pk',
        'pm', 'pp', 'ra', 'rd', 're', 'rf', 'rn', 'rp', 'rr', 'rs', 'sj',
        'sp', 'st'], dtype=object),
 array([77, 77, 76, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77,
        77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77,
        77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 

We see that each speaker has roughly an equal number of utterancesin the train_val set, hence we can do a stratify split by gender into the train-val set. 

In [41]:
train_speakers = df_train[['speaker', 'gender']].drop_duplicates()

In [42]:
print('number of speakers in train-val set', len(list(train_speakers['speaker'])))

number of speakers in train-val set 112


In [43]:
from sklearn.model_selection import train_test_split 
train, val = train_test_split(train_speakers, train_size=0.9, random_state=0, stratify=train_speakers['gender'])

Let's check what the gender distribution is in the train, val set

In [18]:
df_train['mspec_dynamic'].values[0][0]


array([ 0.33635156, -0.21908844, -0.36355532, -0.49843517, -1.39492037,
       -0.18311234, -0.26989952, -0.49467826, -2.19535765, -1.67742775,
       -0.61662301,  0.15560889,  1.79280077,  1.67083037, -0.26102995,
       -1.23877981, -0.10652852,  0.97001643, -0.24019016, -0.08991728,
       -0.65089171, -0.2554614 ,  1.23605617,  1.03063507,  1.36728737,
        0.82889043,  0.18422378,  0.50751303,  1.73446129,  1.70585345,
        1.87938942,  1.01981264,  1.2567082 ,  1.30574304,  1.79602232,
        3.00156044,  2.43952502,  1.94779432,  2.7101488 ,  2.18438797,
        1.19276804,  1.14961276,  0.74028984, -0.97944721, -1.76410695,
       -0.25597138,  0.24631659,  0.48100892, -0.40626892, -0.34258786,
       -1.12367655, -0.72095253, -0.9405887 , -0.91485584, -0.30200621,
        1.05488609,  0.70698156,  0.24464445, -0.07465874,  0.50228125,
        0.57457269,  1.36950666,  0.88794672,  1.47302525,  1.56195474,
        0.68496624,  1.29119423,  1.73719145,  0.78822844,  0.03

In [44]:
print('Train set:', len(train[train['gender']=='man']), 'men vs', len(train[train['gender']=='woman']), 'women')
print('Train set:', len(val[val['gender']=='man']), 'men vs', len(val[val['gender']=='woman']), 'women')

Train set: 49 men vs 51 women
Train set: 6 men vs 6 women


In [45]:
# retreive back full data
train_speakers = list(train['speaker'])
df_val = df_train[~df_train['speaker'].isin(train_speakers)]
df_train = df_train[df_train['speaker'].isin(train_speakers)]

In [46]:
df_train.shape

(7699, 8)

In [47]:
df_val.shape

(924, 8)

In [48]:
df_train.keys()

Index(['filename', 'speaker', 'gender', 'lmfcc_dynamic', 'mspec_dynamic',
       'lmfcc', 'mspec', 'target'],
      dtype='object')

In [ ]:
np.save(f'data/{dataset}_lmfcc.npy', np.array(df['lmfcc']))
np.save(f'data/{dataset}_lmfcc_dynamic.npy', np.array(df['lmfcc_dynamic']))
np.save(f'data/{dataset}_mspec.npy', np.array(df['mspec']))
np.save(f'data/{dataset}_targets.npy', np.array(df['targets']))

In [23]:
np.save(f'data/{dataset}_mspec_dynamic.npy', np.array(df['mspec_dynamic']))

NameError: name 'dataset' is not defined

In [40]:
import pickle
with open('data/train_mspec_dynamic','wb') as f:
    pickle.dump(np.array(df_train['mspec_dynamic']), f)
np.save('data/train_mspec_dynamic.npy', np.array(df_train['mspec_dynamic']))

In [49]:
np.save('data/train_mspec_dynamic.npy', np.array(df_train['mspec_dynamic']))

In [50]:
np.save('data/train_lmfcc_dynamic.npy', np.array(df_train['lmfcc_dynamic']))

In [51]:
np.save('data/train_target_dynamic.npy', np.array(df_train['target']))

In [52]:
np.save('data/val_mspec_dynamic.npy', np.array(df_val['mspec_dynamic']))

In [53]:
np.save('data/val_lmfcc_dynamic.npy', np.array(df_val['lmfcc_dynamic']))

In [54]:
np.save('data/val_target_dynamic.npy', np.array(df_val['target']))

In [17]:
def save_data(df, dataset):

    np.save(f'data/{dataset}_lmfcc.npy', np.array(df['lmfcc']))
    np.save(f'data/{dataset}_lmfcc_dynamic.npy', np.array(df['lmfcc_dynamic']))
    np.save(f'data/{dataset}_mspec.npy', np.array(df['mspec']))
    np.save(f'data/{dataset}_mspec_dynamic.npy', np.array(df['mspec_dynamic']))
    np.save(f'data/{dataset}_targets.npy', np.array(df['targets']))

In [ ]:
save_data(df_train, 'data/train')


In [ ]:
del train_data
del df_train
del df_val


In [4]:
test_data=np.load('data/testdata.npz',allow_pickle=True)['arr_0']

In [7]:
df_test = process_data(test_data)

Computing dynamic features
(91,)


In [8]:
df_test.keys()

Index(['filename', 'speaker', 'gender', 'lmfcc_dynamic', 'mspec_dynamic',
       'lmfcc', 'mspec', 'target'],
      dtype='object')

In [9]:
np.save('data/test_mspec_dynamic.npy', np.array(df_test['mspec_dynamic']))
np.save('data/test_lmfcc_dynamic.npy', np.array(df_test['lmfcc_dynamic']))
np.save('data/test_target_dynamic.npy', np.array(df_test['target']))

In [26]:
df_test['mspec_dynamic'].shape

(8700,)

In [27]:
df_test['mspec_dynamic'][0].shape

AttributeError: 'list' object has no attribute 'shape'

In [1]:
save_data(df_test, 'data/test.npz')

NameError: name 'save_data' is not defined

In [30]:
import numpy as np

def load_data_npz(fpath, feature, dynamic=False):
    data = np.load(fpath, allow_pickle=True)['data'].item()
    if dynamic:
        print('using dynamic features')
        features = data[feature+'_dynamic']
    else:
        print('using ordinary (not dynamic) features')
        features = data[feature]
    targets = np.array(data['targets'])
    return features, targets
    

In [31]:
lmfccs, targets = load_data_npz('data/train.npz', feature='lmfcc', dynamic=True)

using dynamic features


In [23]:
lmfccs[0].shape

(7,)

In [24]:
lmfccs[0][4].shape

(198, 13)

### 4.6 Normalise features 

In [32]:
data_dir = 'data'
dynamic = False
train_lmfccs, train_targets = load_data_npz(data_dir+'/train.npz', feature='lmfcc', dynamic=dynamic)
val_lmfccs, val_targets = load_data_npz(data_dir+'/val.npz', feature='lmfcc', dynamic=dynamic)
test_lmfccs, test_targets = load_data_npz(data_dir+'/test.npz', feature='lmfcc', dynamic=dynamic)

using ordinary (not dynamic) features
using ordinary (not dynamic) features
using ordinary (not dynamic) features


In [38]:
from sklearn.preprocessing import StandardScaler 

def flatten(data):
    flattened_data = np.array([y for x in data for y in x])
    return flattened_data


def scale(train_features, val_features, test_features):
    scaler = StandardScaler()
    scaler.fit(train_features)
    train_features = scaler.transform(train_features)
    val_features = scaler.transform(val_features)
    test_features = scaler.transform(test_features)
    return train_features, val_features, test_features


def normalise_over_train(train_x, val_x, test_x, train_y, val_y, test_y):
    ''' normalise over training set and apply parameters to val, test set'''
    # flatten
    train_x = flatten(train_x)
    val_x = flatten(val_x)
    test_x = flatten(test_x)
    train_y = flatten(train_y)
    val_y = flatten(val_y)
    test_y = flatten(test_y)

    # normalise
    train_x, val_x, test_x = scale(train_x, val_x, test_x)

    return train_x, val_x, test_x, train_y, val_y, test_y


In [39]:
x_train, x_val, x_test, y_train, y_val, y_test = normalise_over_train(train_lmfccs, val_lmfccs, test_lmfccs, \
                                                                      train_targets, val_targets, test_targets)

In [40]:
y_train[0]

39

In [44]:
# one hot encode targets
from keras.utils import np_utils
n_classes = len(np.load('data/StateList.npy'))
print('n classes', n_classes)
y_train = np_utils.to_categorical(np.array(y_train), n_classes)

n classes 61


In [45]:
y_train[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [42]:
x_train.shape

(1342672, 13)

In [41]:
y_train.shape

(1342672,)

In [43]:
x_val.shape

(164720, 13)

In [44]:
y_val.shape

(164720,)

In [45]:
x_test.shape

(1527014, 13)

In [46]:
y_test.shape

(1527014,)